## Import data

In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import boston_housing
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

2025-03-17 09:33:20.666674: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-17 09:33:20.680196: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-17 09:33:20.684429: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-17 09:33:20.694868: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Original data(already divide into training set & test set) 
    'CRIM',     # 城鎮人均犯罪率
    'ZN',       # 佔地面積超過25,000平方呎的住宅用地比例
    'INDUS',    # 城鎮非零售商業用地比例
    'CHAS',     # 查爾斯河虛擬變量（1表示靠近河邊，0表示不靠近）
    'NOX',      # 一氧化氮濃度（百萬分之一）
    'RM',       # 每棟住宅的平均房間數
    'AGE',      # 1940年以前建造的自住單位比例
    'DIS',      # 與波士頓五個就業中心的加權距離
    'RAD',      # 放射性公路的可達性指數
    'TAX',      # 每10,000美元的全額財產稅率
    'PTRATIO',  # 城鎮學生與教師比例
    'B',        # 1000(Bk - 0.63)^2，其中Bk是城鎮黑人比例
    'LSTAT',    # 人口中社會地位較低的百分比

    Target：
    'MEDV'      # 自有住宅的房價中位數，以千美元為單位。

In [2]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()
print(f"Training data:{train_data.shape}")
print(f"Test data:{test_data.shape}")

feature_names = [
    'CRIM',     # 城鎮人均犯罪率
    'ZN',       # 佔地面積超過25,000平方呎的住宅用地比例
    'INDUS',    # 城鎮非零售商業用地比例
    'CHAS',     # 查爾斯河虛擬變量（1表示靠近河邊，0表示不靠近）
    'NOX',      # 一氧化氮濃度（百萬分之一）
    'RM',       # 每棟住宅的平均房間數
    'AGE',      # 1940年以前建造的自住單位比例
    'DIS',      # 與波士頓五個就業中心的加權距離
    'RAD',      # 放射性公路的可達性指數
    'TAX',      # 每10,000美元的全額財產稅率
    'PTRATIO',  # 城鎮學生與教師比例
    'B',        # 1000(Bk - 0.63)^2，其中Bk是城鎮黑人比例
    'LSTAT',    # 人口中社會地位較低的百分比
]

train_df = pd.DataFrame(train_data, columns=feature_names)
print("Training sample:")
for i, name in enumerate(feature_names):
    print(f"{name}: {train_data[0][i]}")

print(f"Targets sample:{train_targets[0]}")

Training data:(404, 13)
Test data:(102, 13)
Training sample:
CRIM: 1.23247
ZN: 0.0
INDUS: 8.14
CHAS: 0.0
NOX: 0.538
RM: 6.142
AGE: 91.7
DIS: 3.9769
RAD: 4.0
TAX: 307.0
PTRATIO: 21.0
B: 396.9
LSTAT: 18.72
Targets sample:15.2


## Feature Normalization
- 對測試資料正規化時，正規化的數值要使用從訓練資料得出來的
- 不能對測試資料做修改

In [3]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

## Q1. 測試模型深度與節點數的影響（使用以下進行排列組合） 
隱藏層數Dense layer: 1, 2, 3, 4, 5, 6, 7  
節點數Node: 16, 32, 64, 128 

固定參數: 
1. 激活函數Activation function: ReLU
2. 世代epoch: 100
3. 批次大小batch size: 16
4. 優化器optimizer: adam

In [4]:
hidden_layers = [1, 2, 3, 4, 5, 6, 7]
hidden_nodes = [16, 32, 64, 128]

results = []

### 建構模型method

In [5]:
def build_model(num_of_layers, num_of_nodes):
    model = keras.Sequential()
    
    model.add(layers.Dense(num_of_nodes, activation='relu'))
    for _ in range(num_of_layers - 1):
        model.add(layers.Dense(num_of_nodes, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer="adam", loss="mse", metrics=["mae", "mse", "mape"])
    model.fit(train_data, train_targets,epochs=100,batch_size=16,validation_split=0.2,verbose=0)
    train_metrics = model.evaluate(train_data, train_targets, verbose=0)
    train_mae = train_metrics[1]
    train_mse = train_metrics[2]
    train_mape = train_metrics[3]
    train_rmse = math.sqrt(train_mse)
    test_metrics = model.evaluate(test_data, test_targets, verbose=0)
    test_mae = test_metrics[1]
    test_mse = test_metrics[2]
    test_mape = test_metrics[3]
    test_rmse = math.sqrt(test_mse)
    return {
        'num_of_layers': num_of_layers,
        'num_of_nodes': num_of_nodes, 
        'train_mae': train_mae, 
        'train_rmse': train_rmse, 
        'train_mape': train_mape,
        'test_mae': test_mae, 
        'test_rmse': test_rmse, 
        'test_mape': test_mape
    }

### 訓練模型(所有排列組合)

In [6]:
for layers_count in hidden_layers:
    for node_count in hidden_nodes:
        print(f"訓練層數：{layers_count}  節點數：{node_count} ")
        result = build_model(layers_count, node_count)
        results.append(result)
        print(result)

訓練層數：1  節點數：16 


I0000 00:00:1742204004.521051   95881 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1742204004.536997   95881 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1742204004.537037   95881 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1742204004.538708   95881 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1742204004.538742   95881 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

{'num_of_layers': 1, 'num_of_nodes': 16, 'train_mae': 3.262439012527466, 'train_rmse': 4.5610150572719155, 'train_mape': 16.36998176574707, 'test_mae': 3.8859498500823975, 'test_rmse': 4.912864357052401, 'test_mape': 21.428674697875977}
訓練層數：1  節點數：32 
{'num_of_layers': 1, 'num_of_nodes': 32, 'train_mae': 2.7678709030151367, 'train_rmse': 3.883825895208171, 'train_mape': 13.986145973205566, 'test_mae': 3.683199405670166, 'test_rmse': 4.855154676399373, 'test_mape': 19.67266082763672}
訓練層數：1  節點數：64 
{'num_of_layers': 1, 'num_of_nodes': 64, 'train_mae': 2.2797229290008545, 'train_rmse': 3.2835301787208095, 'train_mape': 11.450716018676758, 'test_mae': 3.209524393081665, 'test_rmse': 4.680246870588486, 'test_mape': 16.459949493408203}
訓練層數：1  節點數：128 
{'num_of_layers': 1, 'num_of_nodes': 128, 'train_mae': 2.1005043983459473, 'train_rmse': 3.036166578947405, 'train_mape': 10.469404220581055, 'test_mae': 3.112743854522705, 'test_rmse': 4.817397758851992, 'test_mape': 15.573625564575195}
訓練

### Data frame

In [7]:
train_results_df = pd.DataFrame([
    {
        '層數': r['num_of_layers'], '節點數': r['num_of_nodes'], 
        'MAE': r['train_mae'], 'RMSE': r['train_rmse'], 'MAPE': r['train_mape']
    } for r in results
])

test_results_df = pd.DataFrame([
    {
        '層數': r['num_of_layers'], '節點數': r['num_of_nodes'], 
        'MAE': r['test_mae'], 'RMSE': r['test_rmse'], 'MAPE': r['test_mape']
    } for r in results
])

print("\n訓練績效表現:")
print(train_results_df)

print("\n測試績效表現:")
print(test_results_df)


訓練績效表現:
    層數  節點數       MAE      RMSE       MAPE
0    1   16  3.262439  4.561015  16.369982
1    1   32  2.767871  3.883826  13.986146
2    1   64  2.279723  3.283530  11.450716
3    1  128  2.100504  3.036167  10.469404
4    2   16  2.201044  3.116048  11.203763
5    2   32  1.949404  2.825173   9.919171
6    2   64  1.826602  2.707303   9.153197
7    2  128  1.616964  2.463661   8.379547
8    3   16  2.064284  2.909080  10.591621
9    3   32  1.799650  2.687667   9.258734
10   3   64  1.542369  2.414234   7.853667
11   3  128  1.187061  1.860323   5.822185
12   4   16  1.927105  2.869938   9.705064
13   4   32  1.552275  2.353536   8.223660
14   4   64  1.222523  1.950978   6.080668
15   4  128  1.139250  1.937888   5.847597
16   5   16  1.965626  2.873719  10.061539
17   5   32  1.686996  2.561646   9.296263
18   5   64  1.668357  2.431194   7.899959
19   5  128  1.140933  1.667999   5.545121
20   6   16  1.919060  2.637848   9.906807
21   6   32  1.539703  2.326736   7.707614
22

In [12]:
train_results_df.to_excel('Q1-1.xlsx', sheet_name='Q1', index=True, startrow=1, startcol=1)
test_results_df.to_excel('Q1-2.xlsx', sheet_name='Q2', index=True, startrow=1, startcol=8)

### reference
1. boston dataset: https://lib.stat.cmu.edu/datasets/boston
2. Boston Housing Price dataset with Keras(kaggle): https://www.kaggle.com/code/shanekonaung/boston-housing-price-dataset-with-keras